In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

u'E:\\project\\pychram\\book_scripts'

In [2]:
def load_stock_base_msg():
    stock_data_path = 'E:/project/pychram/traderesp/base/output-csv/'
    stock_data_cvs = stock_data_path + 'output-2013-2014-all-stock-base-msg.csv'
    #
    stat = pd.read_csv(stock_data_cvs,encoding='gbk',index_col=['code'])
    return stat

In [23]:
def get_date_period_staticstics(begtime,endtime):
    #
    stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    stock_data_cvs = stock_data_path + 'output-stock-history-adjust-m26-' \
                                     + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    #
    stat = pd.read_csv(stock_data_cvs,parse_dates=['buy_date','sell_date'],encoding='gbk')
    
    #逐笔利润
    stat['profit'] = stat['sell_price'] - stat['buy_price']
    #
    action_num = len(stat)
    suc_num = stat[stat['profit'] > 0.00 ].count() 
    fail_num = action_num - suc_num
    #
    tmp_profit = stat.groupby(['code']).sum()
    year_action_num = len(tmp_profit)
    tmp_profit['profit_rate'] = (tmp_profit['sell_price'] - tmp_profit['buy_price']) / tmp_profit['buy_price']
    year_suc_num = tmp_profit[tmp_profit['profit_rate'] > 0.00].count()
    year_fail_num = year_action_num - year_suc_num
    #
    year_suc_num_gt_percet_5 = tmp_profit[tmp_profit['profit_rate'] > 0.05].count()
    #
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '-----  ',begtime,'  ----  ',endtime,'  -----'
    print '股票总数(排重) :',year_action_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',suc_num['profit']
    print '其中亏损次数 :',fail_num['profit']
    print '获利次数比率 :',suc_num['profit'] / action_num
    print '亏损次数比率 :',fail_num['profit'] / action_num
    print '年度获利股票数 :',year_suc_num['profit_rate']
    print '年度亏损股票数 :',year_fail_num['profit_rate']
    print '年度获利股票数比率 :',year_suc_num['profit_rate'] / year_action_num
    print '年度亏损股票数比率 :',year_fail_num['profit_rate'] / year_action_num
    print '年度获利股票数( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate']
    print '年度获利股票数比率( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate'] / year_action_num
    print '---------------------------------------------------------------------'
    #
    return tmp_profit[tmp_profit['profit_rate'] > 0.00].sort_values('profit_rate',ascending=False),tmp_profit,stat

In [4]:
def show_date_staticstics():
    code1 = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))
    code2 = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))
    code3 = set(code1.index) & set(code2.index)
    year_2_num = len(code3)
    print '连续2年上涨股票数(获利股票数) :',year_2_num
    print '连续2年上涨股票数比率 :',year_2_num / len(code1)
    #--------------------------------------------------------
    code1_gt_percent_5 =  code1[code1['profit_rate'] > 0.05]
    code2_gt_percent_5 =  code2[code2['profit_rate'] > 0.05]
    code3_gt_percent_5 =  set(code1_gt_percent_5.index) & set(code2_gt_percent_5.index)
    print '连续2年上涨股票数(获利股票数 大于 5%) :',len(code3_gt_percent_5)
    print '连续2年上涨股票数比率(获利大于 5%) :',len(code3_gt_percent_5) / len(code1)
    #--------------------------------------------------------
    code4 = code1.index[0:100]
    code5 = code2.index[0:100]
    code6 = set(code4) & set(code5)
    year_2_num_from_100 = len(code6)
    print '连续2年上涨股票数(Top 100) :',year_2_num_from_100
    code7 = code1.index[0:200]
    code8 = code2.index[0:200]
    code9 = set(code7) & set(code8)
    year_2_num_from_200 = len(code9)
    print '连续2年上涨股票数(Top 200) :',year_2_num_from_200
    code10 = code1.index[0:300]
    code11 = code2.index[0:300]
    code12 = set(code10) & set(code11)
    year_2_num_from_300 = len(code12)
    print '连续2年上涨股票数(Top 300) :',year_2_num_from_300
    code13 = code1.index[0:400]
    code14 = code2.index[0:400]
    code15 = set(code13) & set(code14)
    year_2_num_from_400 = len(code15)
    print '连续2年上涨股票数(Top 400) :',year_2_num_from_400
    code16 = code1.index[0:500]
    code17 = code2.index[0:500]
    code18 = set(code16) & set(code17)
    year_2_num_from_500 = len(code18)
    print '连续2年上涨股票数(Top 500) :',year_2_num_from_500
    #
    code_bing = pd.merge(code1,code2,left_index=True, right_index=True)
    return (code_bing,code_bing[(code_bing['profit_rate_x'] > 0.05) & (code_bing['profit_rate_y'] > 0.05)])

In [5]:
code_base_msg = load_stock_base_msg()
code_profit_rate_ge_0, code_profit_rate_ge_5 = show_date_staticstics()
code_profit_rate_ge_5 = code_profit_rate_ge_5[['profit_rate_x','profit_rate_y']]

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 21508
其中获利次数 : 7392
其中亏损次数 : 14116
获利次数比率 : 0.343686070299
亏损次数比率 : 0.656313929701
年度获利股票数 : 1142
年度亏损股票数 : 1326
年度获利股票数比率 : 0.462722852512
年度亏损股票数比率 : 0.537277147488
年度获利股票数( 盈利大于5%) : 96
年度获利股票数比率( 盈利大于5%) : 0.0388978930308
---------------------------------------------------------------------
-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2575
买卖总次数(一次买卖算一次) : 28205
其中获利次数 : 13877
其中亏损次数 : 14328
获利次数比率 : 0.492004963659
亏损次数比率 : 0.507995036341
年度获利股票数 : 1782
年度亏损股票数 : 793
年度获利股票数比率 : 0.692038834951
年度亏损股票数比率 : 0.307961165049
年度获利股票数( 盈利大于5%) : 327
年度获利股票数比率( 盈利大于5%) : 0.126990291262
---------------------------------------------------------------------
连续2年上涨股票数(获利股票数) : 754
连续2年上涨股票数比率 : 0.660245183888
连续2年上涨股票数(获利股票数 大于 5%) : 9
连续2年上涨股票数比率(获利大于 5%) : 0.00788091068301
连续2年上涨股票数(Top 100) : 2
连续2年上涨股票数(Top 200) : 7
连续2年上涨股票数(Top 300) : 20
连续2年上涨股票数(Top 400) : 40

In [ ]:
code_bing = pd.merge(code_base_msg,code_profit_rate_ge_5,left_index=True, right_index=True)

In [ ]:
code_bing.columns

In [ ]:
def trim_float_num(x):
    #print type(x)
    #print 'x=',x
    xx = 0.000
    if x is None:
        xx = 0.000
    else:
        xx = float(str(x))
    # print xx
    if xx < 0.001:
        return "0.1 %"
    else:
        ret = '%.1f' %(xx * 100.00)
        return ret + ' %'

In [ ]:
trim_profit_rate_x = code_bing['profit_rate_x'].map(trim_float_num)
code_bing['profit_rate_x'] = trim_profit_rate_x
trim_profit_rate_y = code_bing['profit_rate_y'].map(trim_float_num)
code_bing['profit_rate_y'] = trim_profit_rate_y

In [ ]:
code_bing

In [26]:
code1,code_2,code_all = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 21508
其中获利次数 : 7392
其中亏损次数 : 14116
获利次数比率 : 0.343686070299
亏损次数比率 : 0.656313929701
年度获利股票数 : 1142
年度亏损股票数 : 1326
年度获利股票数比率 : 0.462722852512
年度亏损股票数比率 : 0.537277147488
年度获利股票数( 盈利大于5%) : 96
年度获利股票数比率( 盈利大于5%) : 0.0388978930308
---------------------------------------------------------------------


In [30]:
code_all[code_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
325,2013-02-18,37864.797560,100,378.647976,858,2013-02-22,38721.288375,100,387.212884,8.564908
2731,2013-03-01,36903.183318,100,369.031833,858,2013-03-04,36362.256456,100,363.622565,-5.409269
5128,2013-03-25,34559.160354,100,345.591604,858,2013-03-27,33507.344999,100,335.073450,-10.518154
7523,2013-05-07,34363.793698,100,343.637937,858,2013-06-07,34589.139512,100,345.891395,2.253458
9928,2013-07-17,31320.121269,100,313.201213,858,2013-07-19,30043.659423,100,300.436594,-12.764618
12286,2013-07-24,30059.214569,100,300.592146,858,2013-07-30,29638.947449,100,296.389474,-4.202671
14641,2013-08-02,29965.847459,100,299.658475,858,2013-10-18,28066.689769,100,280.666898,-18.991577
17005,2013-10-22,28237.928886,100,282.379289,858,2013-10-24,27786.500579,100,277.865006,-4.514283
19393,2013-11-19,26152.013448,100,261.520134,858,2013-12-17,25000.012524,100,250.000125,-11.520009


In [31]:
code2,code21,code2_all = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))

-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2575
买卖总次数(一次买卖算一次) : 28205
其中获利次数 : 13877
其中亏损次数 : 14328
获利次数比率 : 0.492004963659
亏损次数比率 : 0.507995036341
年度获利股票数 : 1782
年度亏损股票数 : 793
年度获利股票数比率 : 0.692038834951
年度亏损股票数比率 : 0.307961165049
年度获利股票数( 盈利大于5%) : 327
年度获利股票数比率( 盈利大于5%) : 0.126990291262
---------------------------------------------------------------------


In [33]:
code2_all[code2_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
311,2014-02-11,23272.180995,100,232.721810,858,2014-02-26,23505.669612,100,235.056696,2.334886
2688,2014-03-04,24751.011316,100,247.510113,858,2014-03-05,25171.308239,100,251.713082,4.202969
5054,2014-03-25,26852.510274,100,268.525103,858,2014-03-31,25949.657771,100,259.496578,-9.028525
7389,2014-04-02,26899.220283,100,268.992203,858,2014-04-22,26603.453464,100,266.034535,-2.957668
9708,2014-04-23,27553.010528,100,275.530105,858,2014-04-24,27817.657194,100,278.176572,2.646467
12065,2014-05-27,25871.872212,100,258.718722,858,2014-05-28,26214.338184,100,262.143382,3.424660
14373,2014-05-29,26136.507814,100,261.365078,858,2014-06-05,26198.762473,100,261.987625,0.622547
16674,2014-06-06,26385.559649,100,263.855596,858,2014-06-20,26525.653681,100,265.256537,1.400940
18999,2014-07-01,27568.631032,100,275.686310,858,2014-07-10,28719.339690,100,287.193397,11.507087
21318,2014-07-15,31134.213962,100,311.342140,858,2014-08-29,30356.271578,100,303.562716,-7.779424


In [34]:
code21.ix[858]

buy_money      324981.562031
buy_num          1200.000000
buy_price        3249.815620
sell_money     330302.710984
sell_num         1200.000000
sell_price       3303.027110
profit             53.211490
profit_rate         0.016374
Name: 858, dtype: float64